# Git From Scratch

In this project, I try to build Git from scratch.

In [ ]:
# imports
import argparse
import configparser
from datetime import datetime
import grp, pwd
from fnmatch import fnmatch
import hashlib
from math import ceil
import os
import re
import sys
import zlib

In [ ]:
# Parser and Main function
argparser = argparse.ArgumentParser(description='The stupidest content tracker')
argsubparsers = argparser.add_subparsers(title='Commands', dest='command')
argsubparsers.required = True

def main(argv=sys.argv[1:]):
    args = argparser.parse_args(argv)
    match args.command:
        case 'cat-file'    : cmd_cat_file(args)
        case 'init'        : cmd_init(args)
        case 'hash-object' : cmd_hash_object(args)


To deal with Git repositories, we create a `GitRepository` class, which will serve as the basic abstraction to interact with Git repositories.

In [ ]:
class GitRepository(object):
    """A Git Repository"""

    worktree = None
    gitdir = None
    conf = None

    def __init__(self, path, force=False):
        self.worktree = path
        self.gitdir = os.path.join(path, '.git')

        # Check if the repository is a git repository ( except when force is True )
        if not (force or os.path.isdir(self.gitdir)):
            raise Exception(f"Not a git repository {path}")

        # Git has a configuration file in .git. Read those configs        
        self.conf = configparser.ConfigParser()
        cf = repo_file(self, 'config')

        if cf and os.path.exists(cf):
            self.conf.read([cf])
        
        # If force, then continues. Else, raise an exception if config is not there
        elif not force:
            raise Exception("Config file is missing")
        
        if not force:
            vers = int(self.conf.get('core', 'repositoryformatversion'))

            # Git requires the repository format version to be zero
            if vers !=0:
                raise Exception(f"Unsupported repositoryformatversion: {vers}")
            

Here we define some utility functions that deal with directory and file creations in a repository.

In [ ]:
def repo_path(repo:GitRepository, *path) -> str:
    """
    *path -> makes the function variadic, so it can be called with multiple path
            components as separate arguments. For example, 
            repo_path(repo, "objects", "df", "<some hash>") is a valid call. 
            The function receives path as a list 
    """
    return os.path.join(repo.gitdir, *path)

def repo_file(repo:GitRepository, *path, mkdir=False):
    """
    Same as repo_path, but create dirname(*path) if absent.  For example, 
    repo_file(r, \"refs\", \"remotes\", \"origin\", \"HEAD\") will create
    .git/refs/remotes/origin.
    That is, this treats the last entry in the path as file, and creates 
    intermediate dirs if required.
    """

    if repo_dir(repo, *path[:-1], mkdir=mkdir):
        return repo_path(repo, *path)

def repo_dir(repo:GitRepository, *path, mkdir=False):
    """
    Same as repo_path, but mkdir *path if absent if mkdir. Treats the entire path
    as a directory. Creates intermediate directories if required.
    """

    path = repo_path(repo, *path)

    if os.path.exists(path):
        if (os.path.isdir(path)):
            return path
        else:
            raise Exception(f"Not a directory {path}")

    if mkdir:
        os.makedirs(path)
        return path
    else:
        return None

The code to actually create the git repository at a given path, and the command line command handler. That is, we need to be able to pass `wyag init <path>` and it should work.

In [ ]:
    
def repo_create(path):
    """Creates a Git repository at the given path"""

    # The first time you create a repo. You create using force.
    repo = GitRepository(path=path, force=True)

    # The path either should not exist or be an empty git directory
    if os.path.exists(repo.worktree):

        if not os.path.isdir(repo.worktree):  # repo.worktree = path in args
            raise Exception(f"{path} is not a directory")
        # The .git directory needs to be empty
        if os.path.exists(repo.gitdir) and os.listdir(repo.gitdir):
            raise Exception(f"{path} us not empty")
        
    else:
        os.makedirs(repo.worktree)
        
    # When you initialize a git repo, you create some additional folders / files.
    # These dirs are created here.
    assert repo_dir(repo, 'branches', mkdir=True)
    assert repo_dir(repo, 'objects', mkdir=True)
    assert repo_dir(repo, 'refs', 'tags', mkdir=True)
    assert repo_dir(repo, 'refs', 'heads', mkdir=True)

    # The repos description, head, and config files
    with open(repo_file(repo, 'description'), 'w') as f:
        f.write("Unnamed repository; edit this file-'description'- to name the repo.")

    with open(repo_file(repo, 'HEAD'), 'w') as f:
        f.write("ref: refs/heads/master\n")

    with open(repo_file(repo, 'config'), 'w') as f:
        config = repo_default_config()
        config.write(f)

    return repo

def repo_default_config():
    """
    Git config is a simple .ini file, with a [core] tag which has three (or 4) 
    fields. Here, we're supporting the bare minimum configs. Git supports 
    some additional config also.
    """
    ret = configparser.ConfigParser()

    ret.add_section("core")
    ret.set("core", "repositoryformatversion", "0")
    ret.set("core", "filemode", "false")
    ret.set("core", "bare", "false")

    return ret

# The init command: wyag init [path]. Works like: git init [path]

argsp = argsubparsers.add_parser('init', help='Initialize a new, empty repository')
argsp.add_argument('path', metavar='directory', nargs='?', default='.', 
                   help='Where to create the repository.')

# The bridge function that calls the repo create function when the init command 
# is called
def cmd_init(args):
    repo_create(args.path)

A helper function to find out the base directory of the repository.

In [ ]:
def repo_find(path='.', required=True):
    """
    A helper function to find the .git directory of the current repository
    For example, the repo may be at "/home/myproject"
    but you may be working in "/home/myproject/src/api/some_api.py"

    So from the file path, we need to find the root directory for the repo.
    """
    path = os.path.realpath('.')

    if os.path.isdir(os.path.join(path, ".git")):
        return GitRepository(path)


    # If we haven't returned, recurse in parent
    parent = os.path.realpath(os.path.join(path, ".."))

    if parent == path:
        # Raise the exception when no repo was find ( after recursing till '/')
        if required:
            raise Exception("No git directory.")
        else:
            return None

    # Recursive case
    return repo_find(parent, required)

## git hash-object & git cat-file

`git hash-object`: converts an existing file into a git 'object'
`git cat-file`: prints an existing git object to standard output


## Git Objects:
In git, the filename is mathematically related to the content that is there in the file (unlike your filesystem). If even a byte changes in the file, the SHA-1 hash will change and the filename will change. That is to say: when you modify a file, you're not really modifying the file. You are creating a new file at a different location. 

Git objects are the files in the git repository whose path is determined by their contents.

Note: Git is not a key:value store but similar. It's keys are derived from values. Thus, it is more of a value:value pairs.

In git, many things are stored as objects: files, commits, tags. Almost everything is an object in Git.

SHA-1 Hash Structure in Git:
1. First two chars are for the directory
2. Rest of the chars are the filenames

Note:
1. Each character in the SHA-1 string is a hexadecimal value. That means the first two chars can represent 256 possible values. 
2. So Git creates 256 directories- starting from 00 to ff (as required). These directories are NOT what are there in your project. These directories are internal to git to improve the speed of searching. Having this structure ensures that on average most files are distributed across the 256 directories and any one directory doesn't get too big. Thus, improving speed and performance. That is, if you have two files in two different directories of your repo, they cna end up in a same directory inside git! Git doesn't care about how you are organizing your code. 

Run the following command to see the directories stored by Git. Each directory contains files that you have added.
```shell
ls -al .git/objects
```

If we want to create such a feature, we need to know how the files are stored in the directories inside git. 

Storage Format in Git:

Let's say you do `cat .git/objects/<<dir>>/<<hashvalue>>`. You're going to see something like this:

```
00000000  63 6f 6d 6d 69 74 20 31  30 38 36 00 74 72 65 65
```

Let's decode this. All this is hexadecimal. 

1. The first few hexes are offsets. For example: `00000000`. Ignore this for now.
2. The string `commit` in hexadecimal is: `63 6f 6d 6d 69 74`. This is what you see in the first line's first few bytes. That is, the first few bytes give the type of the object: commit, tree, blob, or tag.
3. The object type is followed by a ASCII space character: `0x20`, which indicates the end of the object type.
4. Then we are followed by the size of the file in bytes in hexadecimal. For example, in the above line:`31  30 38 36` is the size of the file. This can vary offcourse. So it is followed by a null character `0x00` in hex. This indicates the end of file size. 
5. Whatever follows this is the actual content of the object.
6. The objects (headers and contents) are stored compressed with zlib.

Let's create an abstract class for the GitObject, which we can extend to create TreeObject, BlobObject, etc.

In [ ]:
class GitObject (object):

    def __init__(self, data=None): # data is supposed to be the compressed hex representatino of object's contents
        if data != None:
            self.deserialize(data)
        else:
            self.init()

    def serialize(self, repo):
        """This function MUST be implemented by subclasses.
 
        It must read the object's contents from self.data, a byte string, and
        do whatever it takes to convert it into a meaningful representation.
        What exactly that means depend on each subclass.

        """
        raise Exception("Unimplemented!")

    def deserialize(self, data):
        raise Exception("Unimplemented!")

    def init(self):
        pass # Just do nothing. This is a reasonable default!

From a given SHA-1 hash value, how do you get the file path?

For example: the path to `e673d1b7eaa0aa01b5bc2442d570a765bdaae751` is `.git/objects/e6/73d1b7eaa0aa01b5bc2442d570a765bdaae751`.

The first two characters become the dictionary in `.git/objects/`, and the remaining string stays the file name.

The file needs to be read as a binary and then decompressed using `zlib`. Then we extract the type header, size, and content by simple string parsing.

In [ ]:
def object_read(repo, sha):
    path = repo_file(repo, 'objects', sha[0:2], sha[2:])

    if not os.path.isfile(path):
        return None
    
    with open(path, 'rb') as f:
        raw = zlib.decompress(f.read())

        x = raw.find(b' ') # Find the ASCII space character's index
        fmt = raw[0:x]

        y = raw.find(b'\x00', x) # find null character after the space character i.e. x

        # Validate whether the size mentioned in the object is the same as what's the size of the file
        size = int(raw[x:y].decode("ascii"))
        if size != len(raw)-y-1:
            raise Exception(f"Malformed object {sha}: bad length")


        # Initialize the appropriate Git Object type based on the type header
        match fmt:
            case b'commit' : c=GitCommit
            case b'tree'   : c=GitTree
            case b'tag'    : c=GitTag
            case b'blob'   : c=GitBlob
            case _:
                raise Exception(f"Unknown type {fmt.decode("ascii")} for object {sha}")

        # Call constructor and return object with its sontents i.e raw[y+1:]
        return c(raw[y+1:])

We also need to be able to write objects. Writing is reverse of reading: We compute the hash, insert the headers, compress everything and write the object in the correct location.

The object consists of everything: the headers, size, and the contents. So you need to compute hash after you have added the headers, etc. 

In [ ]:
def object_write(obj:GitObject, repo=None):

    # Serialize object data
    data = obj.serialize()

    # Add headers and the size
    result = obj.fmt + b' ' + str(len(data)).encode() + b'\x00' + data

    # Compute hash of the data
    sha = hashlib.sha1(result).hexdigest()

    if repo:
        # Compute path
        path=repo_file(repo, "objects", sha[0:2], sha[2:], mkdir=True)

        # You cannot overwrite the files in git. When you modify a file, you create a new file. That's why we have this check before writing.
        if not os.path.exists(path):
            with open(path, 'wb') as f:
                # Compress and write
                f.write(zlib.compress(result))
    return sha

## Blobs

Blobs are simplest in Git. Blobs contain whatever we as programmers write. It's all the code, README, images, etc. There is no serialize / deserialize here. Why? 

<!-- TODO: Need to come back to this point of figuring out what serialize / deserialize, etc. means. And why we don't need to do this with GitBlob -->

In [ ]:
class GitBlob(GitObject):
    fmt =b'blob'

    def serialize(self):
        return self.blobdata
    
    def deserialize(self, data):
        self.blobdata = data

## cat-file Command

`cat-file` command simply prints the deserialized content of the object to stdout without the headers. We need to pass to this command which type of the object this is (blob, tree, commit, or tag). We also need to provide the hash of the file so that the we can find out the location and then print the output to stdout.

In [ ]:
argsp = argsubparsers.add_parser('cat-file', help='Provide the content of repository objects')

argsp.add_argument('type', metavar='type', choices=['blob', 'commit', 'tag', 'tree'], help='Specify the type')
argsp.add_argument('object', metavar='object', help='The object to display')

Now we can create the command bridge function that can handle these commands.

In [ ]:
def cmd_cat_file(args):
    repo = repo_find()
    cat_file(repo, args.object, fmt=args.type.encode())

def cat_file(repo, obj, fmt=None):
    obj = object_read(repo, object_find(repo, obj, fmt=fmt))
    sys.stdout.buffer.write(obj.serialize())

Git has a lot of ways to refer to objects: by hash, by name, by refs, tags, etc. So we define a helper function which we will modify as we progress to find out the object from the different ways of accessing the object. In a way, this is resolving the object.

In [ ]:
# This is a placeholder function that will be updated later. Right now, this is just accessing the object by its hash.
def object_find(repo, name, fmt=None, follow=True):
    return name

## hash-object

While `cat-file` finds the file from the hash and reads the content, we need some way of creating hash from the object content. That's what `hash-object` does. Given an object data, it converts it into a hash and gets a name. We are also keeping a flag (`-w`) to decide whether to store the file or just print the hash on stdout.

In [ ]:
argsp = argsubparsers.add_parser(
    "hash-object",
    help="Compute object ID and optionally creates a blob from a file")

argsp.add_argument("-t",
                   metavar="type",
                   dest="type",
                   choices=["blob", "commit", "tag", "tree"],
                   default="blob",
                   help="Specify the type")

argsp.add_argument("-w",
                   dest="write",
                   action="store_true",
                   help="Actually write the object into the database")

argsp.add_argument("path",
                   help="Read object from <file>")

In [ ]:
def cmd_hash_object(args):
    if args.write:
        repo = repo_find()
    else:
        repo = None

    with open(args.path, "rb") as fd:
        sha = object_hash(fd, args.type.encode(), repo)
        print(sha)

In [ ]:
def object_hash(fd, fmt, repo=None):
    """ Hash object, writing it to repo if provided."""
    data = fd.read()

    # Choose constructor according to fmt argument
    match fmt:
        case b'commit' : obj=GitCommit(data)
        case b'tree'   : obj=GitTree(data)
        case b'tag'    : obj=GitTag(data)
        case b'blob'   : obj=GitBlob(data)
        case _: raise Exception(f"Unknown type {fmt}!")

    return object_write(obj, repo)

## Commits in Git

Commits again are just objects in Git. So when we're dealing with commit as objects in git, they will have the type header, size, followed by the content of the object. Assume that we extracted out the headers and the size. What does the content of the object look like?

```shell
tree 29ff16c9c14e2652b22f8b78bb08a5a07930c147 # key value pairs that are not followed by key value pairs
parent 206941306e8a8af65b66eaaaea388a7ae24d49a0
author Thibault Polge <thibault@thb.lt> 1527025023 +0200
committer Thibault Polge <thibault@thb.lt> 1527025044 +0200
gpgsig -----BEGIN PGP SIGNATURE-----

 iQIzBAABCAAdFiEExwXquOM8bWb4Q2zVGxM2FxoLkGQFAlsEjZQACgkQGxM2FxoL
 kGQdcBAAqPP+ln4nGDd2gETXjvOpOxLzIMEw4A9gU6CzWzm+oB8mEIKyaH0UFIPh
 rNUZ1j7/ZGFNeBDtT55LPdPIQw4KKlcf6kC8MPWP3qSu3xHqx12C5zyai2duFZUU
 wqOt9iCFCscFQYqKs3xsHI+ncQb+PGjVZA8+jPw7nrPIkeSXQV2aZb1E68wa2YIL
 3eYgTUKz34cB6tAq9YwHnZpyPx8UJCZGkshpJmgtZ3mCbtQaO17LoihnqPn4UOMr
 V75R/7FjSuPLS8NaZF4wfi52btXMSxO/u7GuoJkzJscP3p4qtwe6Rl9dc1XC8P7k
 NIbGZ5Yg5cEPcfmhgXFOhQZkD0yxcJqBUcoFpnp2vu5XJl2E5I/quIyVxUXi6O6c
 /obspcvace4wy8uO0bdVhc4nJ+Rla4InVSJaUaBeiHTW8kReSFYyMmDCzLjGIu1q
 doU61OM3Zv1ptsLu3gUE6GU27iWYj2RWN3e3HE4Sbd89IFwLXNdSuM0ifDLZk7AQ
 WBhRhipCCgZhkj9g2NEk7jRVslti1NdN5zoQLaJNqSwO1MtxTmJ15Ksk3QP6kfLB
 Q52UWybBzpaP9HEd4XnR+HuQ4k2K0ns2KgNImsNvIyFwbpMUyUWLMPimaV1DWUXo
 5SBjDB/V/W2JBFR+XKHFJeFwYhj7DD/ocsGr4ZMx/lgc8rjIBkI=
 =lgTX
 -----END PGP SIGNATURE-----

Create first draft # Commit message followed by a newline and preceded by a newline

```

Note the format: Initial few lines are key-value pairs which are separated by space. Then we have a newline, followed by the commit message, followed by another newline. We need to write a parser for this also. That is, we need to write the GitCommit class.

For a commit object, we have the following fields:

1. `tree`: Which I think has the hash value of the entire working directory. It's everything- all the files, their contents, etc.
2. `parent`: Which commit is the parent of this commit. There may be more than one (e.g. when branching) or there may be none (e.g. when creating a new repo)
3. `author`: Who wrote the commit
4. `committer`: Who committed the commit. In collaborative environment, author and committer may be different.
5. `gpgsig`: I think it's a PGP signature to verify that the contents haven't been tampered with

Each of the above key value pairs can span multiple lines. If a value is spanning multiple lines, then those continuation lines are marked by a starting space character followed by the content. So the true new lines, are the newlines not followed by a space. But if it's a continuation line, then newline is followed by a space character

If we want to have functionality that lets us interact with commits, we would need to parse the contents fo the `commit` objects in Git.

**Note:** The parser for the `commit` object and the `tag` object is the same. So we will write only one generic parser for both.


Now, let's think:

1. When will we have the message? When we first encounter a blank line. That is, the newline character comes before the space character. Whatever follows after this empty line is the commit message.

2. When will we have key-value pairs and how to parse them? Note that this is compressed


I have pretty much copy pasted the code below. But it is not that difficult to work out since it's just string parsing based on the above mentioned format.

In [ ]:
def kvlm_parse(raw, start=0, dct=None): # key-value list with Message Parser
    if not dct:
        # Since we have recursive calls, we cannot initialize dct=dict() because it will cause infinite looping
        # In each recursive call, we are reading one key-value pair and moving to the next line
        dct = dict()

    # This function is recursive: it reads a key/value pair, then call
    # itself back with the new position.  So we first need to know
    # where we are: at a keyword, or already in the messageQ

    # We search for the next space and the next newline.
    spc = raw.find(b' ', start)
    nl = raw.find(b'\n', start)

    # If space appears before newline, we have a keyword.  Otherwise,
    # it's the final message, which we just read to the end of the file.

    # Base case
    # =========
    # If newline appears first (or there's no space at all, in which
    # case find returns -1), we assume a blank line.  A blank line
    # means the remainder of the data is the message.  We store it in
    # the dictionary, with None as the key, and return.
    if (spc < 0) or (nl < spc):
        assert nl == start
        dct[None] = raw[start+1:]
        return dct

    # Recursive case
    # ==============
    # we read a key-value pair and recurse for the next.
    key = raw[start:spc]

    # Find the end of the value.  Continuation lines begin with a
    # space, so we loop until we find a "\n" not followed by a space.
    end = start
    while True:
        end = raw.find(b'\n', end+1)
        if raw[end+1] != ord(' '): break

    # Grab the value
    # Also, drop the leading space on continuation lines
    value = raw[spc+1:end].replace(b'\n ', b'\n')

    # Don't overwrite existing data contents
    if key in dct:
        if type(dct[key]) == list:
            dct[key].append(value)
        else:
            dct[key] = [ dct[key], value ]
    else:
        dct[key]=value

    return kvlm_parse(raw, start=end+1, dct=dct)

## Git Object Identity Rules



## Serializing Git Tree

Again, this is just copy pasted from the repo as it's justing converting the tree dictionary into bytes based on some format.

One important thing to note is that the dictionary in Python is ordered. This is important because if we change the order of the contents of tree (which is basically key value pairs), then we're going to change the hash of the tree. So it's an important assumption that the order is being maintained. 

In [ ]:
def kvlm_serialize(kvlm):
    ret = b''

    # Output fields
    for k in kvlm.keys():
        # Skip the message itself
        if k == None: continue
        val = kvlm[k]
        # Normalize to a list
        if type(val) != list:
            val = [ val ]

        for v in val:
            ret += k + b' ' + (v.replace(b'\n', b'\n ')) + b'\n'

    # Append message
    ret += b'\n' + kvlm[None]

    return ret

## Git Log

Git Log is a extremely rich command which provides visualization, etc. Here we use graphviz for visualization, and implement a much simpler `git log` command.

In [ ]:
argsp = argsubparsers.add_parser('log', help='Display history of a given commit')

argsp.add_argument('commit', default='HEAD', nargs='?', help='commit to start at')

In [ ]:
def cmd_log(args):
    repo = repo_find()

    print("digraph wyaglog{")
    print("  node[shape=rect]")
    log_graphviz(repo, object_find(repo, args.commit), set())
    print("}")

def log_graphviz(repo, sha, seen):

    if sha in seen:
        return
    seen.add(sha)

    commit = object_read(repo, sha)
    message = commit.kvlm[None].decode("utf8").strip()
    message = message.replace("\\", "\\\\")
    message = message.replace("\"", "\\\"")

    if "\n" in message: # Keep only the first line
        message = message[:message.index("\n")]

    print(f"  c_{sha} [label=\"{sha[0:7]}: {message}\"]")
    assert commit.fmt==b'commit'

    if not b'parent' in commit.kvlm.keys():
        # Base case: the initial commit.
        return

    parents = commit.kvlm[b'parent']

    if type(parents) != list:
        parents = [ parents ]

    for p in parents:
        p = p.decode("ascii")
        print (f"  c_{sha} -> c_{p};")
        log_graphviz(repo, p, seen)

You can now use our log command like this:
```shell
wyag log e03158242ecab460f31b0d6ae1642880577ccbe8 > log.dot
dot -O -Tpdf log.dot
```

## Commits as a Directed Acyclic Graph

We can think of each commit as a node in a commit graph. Each node has a directed edge back to its parent. Each node can have have zero or more parents.

Each commit is made of the following things: 
1. author & timestamp
2. committer & timestamp
3. a tree object (which contains hashes of contents)
4. an optional PGP signature
5. parent (which recursively is bound to *all* parents). 
6. A message

If any of these change, then that changes the hash! So we will be creating some new object in git. 

That's why objects are immutable in git.

## Trees in Git

Each commit file has a tree in it. So what does a tree contain?

Tree is again a git object which is identified by it's hash. Note that even though the objects are different, to git, the software, they are just objects which it stores as per the hashing convention we discussed above. So a tree object also will have first two characters specifying the directory and the rest of the characters specifying the ID.

If we go to the directory and do `cat` on the tree file, we may get something like this:

```
100644	894a44cc066a027465cd26d634948d56d13af9af    .gitignore
100644	94a9ed024d3859793618152ea559a168bbcbb5e2	LICENSE
100644	bab489c4f4600a38ce6dbfd652b90383a4aa3e45	README.md
100644	6d208e47659a2a10f5f8640e0155d9276a2130a9	src
040000	e7445b03aea61ec801b20d6ab62f076208b7d097	tests
040000	d5ec863f17f3a2e92aa8f6b66ac18f7b09fd1b38	main.c
```

1. First part is the mode of the file- whether it can be read, written, or executed. 
2. Second part is the hash of the file. 
3. Third part is the filename.
4. If it's a directory, then it's treated as another tree object. 


So what we are doing here is the following:

1. Commit stores hash of the tree. 
2. Tree stores the hashes of the files and directories (which are treated as trees) at the time of the commit along with their names and modes.
3. Using those hashes, we can find out the contents of the object using the naming conventions. That is, in the above example, if go to the following path: `.git/objects/89/a44cc066a027465cd26d634948d56d13af9af`, we are going to get the compressed contents of the `.gitignore` file at the time of the commit.  

The path identifies exactly one file or directory. If you have five levels of nested directories, even if four are empty save the next directory, you’re going to need five tree objects recursively referring to one another. You cannot take the shortcut of putting a full path in a single tree entry, like `dir1/dir2/dir3/dir4/dir5`.

The format of contents of a tree:

`[mode] space [filename or path] 0x00 [sha-1 hash]`

1. Mode: upto six bytes, and an octal repr of the mode stored in ASCII.
2. Followed by space: ASCII `0x20`
3. Followed by a path which is terminated by a null character `0x00`.
4. SHA-1 in binary encoding in 20 bytes.

**Note:** The order of the files stored in the tree object matters! If the order changes, the hash changes. So even if we have the same files but they are present in the different order inside the tree file, git is going to treat it as a separate object. So to avoid doing this, we simply sort the objects. How do we sort the entries in a tree? Git does it as follows:
1. Regular files are sorted in alphabetical order
2. Directories are sorted after adding `/` at the end.

It matters, because it means that if `whatever` names a regular file, it will sort before `whatever.c`, but if `whatever` is a dir, it will sort *after*, as `whatever/`.

Now let's parse the trees. We write the parser in two stages: one, the *leaf* that will hold one entry in the tree file. Then we write the tree container that will hold all the leaves of the tree.

To hold this data inside our program, we create a `GitTreeLeaf` class.

In [ ]:
class GitTreeLeaf (object):
    def __init__(self, mode, path, sha):
        self.mode = mode
        self.path = path
        self.sha = sha

In [ ]:
def tree_parse_one(raw, start=0):
    """
    Return the `leaf` and also return the position in the file from which
    the real parser has to continue parsing for more leaves.
    """

    # Find the space terminator of the mode
    x = raw.find(b' ', start)
    assert x-start == 5 or x-start==6

    # Read the mode
    mode = raw[start:x]
    if len(mode) == 5:
        # Normalize to six bytes.
        mode = b"0" + mode

    # Find the NULL terminator of the path
    y = raw.find(b'\x00', x)
    # and read the path
    path = raw[x+1:y]

    # Read the SHA…
    raw_sha = int.from_bytes(raw[y+1:y+21], "big")
    # and convert it into an hex string, padded to 40 chars
    # with zeros if needed.
    sha = format(raw_sha, "040x")
    return y+21, GitTreeLeaf(mode, path.decode("utf8"), sha)

def tree_parse(raw):
    "Parse the entire tree and store it as a leaf"
    pos = 0
    max = len(raw)
    ret = list()
    while pos < max:
        pos, data = tree_parse_one(raw, pos)
        ret.append(data)

    return ret

The sorting function to sort the entries in the tree file. Once we have the entries in the sorted order, we can compress and write them to the tree file when we serialize the tree.

In [ ]:
def tree_leaf_sort_key(leaf):
    if leaf.mode.startswith(b"10"):
        return leaf.path
    else:
        return leaf.path + "/"

def tree_serialize(obj):
    obj.items.sort(key=tree_leaf_sort_key)
    ret = b''
    for i in obj.items:
        ret += i.mode
        ret += b' '
        ret += i.path.encode("utf8")
        ret += b'\x00'
        sha = int(i.sha, 16)
        ret += sha.to_bytes(20, byteorder="big")
    return ret

In [ ]:
# An abstraction: a Tree class to wrap the above tree functions
class GitTree(GitObject):
    fmt=b'tree'

    def deserialize(self, data):
        self.items = tree_parse(data)

    def serialize(self):
        return tree_serialize(self)

    def init(self):
        self.items = list()

## Checkout Command

Now, if you think from an implementation perspective, what does the command `git checkout` do? It goes to a particular a commit (be it a commit or a branch), and then instantiates that commit. And what does instantiation mean? Instantiation basically means creating a copy of that commit. So in terms of the filesystem, the checkout command will take the commit hash and a directory as arguments. Then it will instantiate the tree from the hash and save it as a copy in the directory. We don't want to overwrite if there is any data already present in the directory. So our command only works if the directory is empty.

In [ ]:
argsp = argsubparsers.add_parser("checkout", help="Checkout a commit inside of a directory.")

argsp.add_argument("commit",
                   help="The commit or tree to checkout.")

argsp.add_argument("path",
                   help="The EMPTY directory to checkout on.")

In [ ]:
# Wrapper function
def cmd_checkout(args):
    repo = repo_find()

    obj = object_read(repo, object_find(repo, args.commit))

    # If the object is a commit, we grab its tree
    if obj.fmt == b'commit':
        obj = object_read(repo, obj.kvlm[b'tree'].decode("ascii"))

    # Verify that path is an empty directory
    if os.path.exists(args.path):
        if not os.path.isdir(args.path):
            raise Exception(f"Not a directory {args.path}!")
        if os.listdir(args.path):
            raise Exception(f"Not empty {args.path}!")
    else:
        os.makedirs(args.path)

    tree_checkout(repo, obj, os.path.realpath(args.path))

In [ ]:
# the function that writes the copy in the directory.
def tree_checkout(repo, tree, path):
    for item in tree.items:
        obj = object_read(repo, item.sha)
        dest = os.path.join(path, item.path)

        if obj.fmt == b'tree':
            os.mkdir(dest)
            tree_checkout(repo, obj, dest)
        elif obj.fmt == b'blob':
            # @TODO Support symlinks (identified by mode 12****)
            with open(dest, 'wb') as f:
                f.write(obj.blobdata)

## Refs, Tags, and Branches






























